# exercise 1: protein-ligand separation

In [3]:
import numpy as np

In [4]:
input_file = 'pdb/3HMM.pdb'
fp = open(input_file)
lines = fp.readlines()
fp.close()

chain_dict = dict()
chain_seq_dict = dict()
chain_ligand_dict = dict()
conect_dict = dict()
mod_res_dict = dict()

check = False
protein_line_list = list()
ligand_line_dict = dict()

conect_dict = dict()

for line in lines:
    if line[0:10] == 'REMARK 465':
        if line[15:27] == 'RES C SSSEQI':
            check = True
            continue
        if not check :
            continue
        res = line[15:18]
        chain_id = line[19]
        res_num = line[22:26]
        insertion_code = line[26]
        res_num2 = line[22:27]

        if chain_id not in chain_dict:
            chain_dict[chain_id] = dict()

        chain_dict[chain_id][res_num2] = res
    if line[0:6] == 'MODRES':
        res_name_mod = line[12:15]
        chain_id = line[17]
        res_num = line[18:22]
        res_num2 = line[18:23]
        res_name_o = line[24:27]
        if chain_id not in mod_res_dict :
            mod_res_dict[chain_id] = dict()
        if res_name_mod not in mod_res_dict[chain_id]:
            mod_res_dict[chain_id][res_name_mod] = list()
        mod_res_dict[chain_id][res_name_mod] += [[res_num2,res_name_o]]
        
    if line[0:6] == 'ATOM  ':
        res = line[17:20]
        chain_id = line[21]
        res_num = line[22:26]
        res_num2 = line[22:27]
        atom_type = line[12:16]

        protein_line_list += [line]
        
    if line[0:6] == 'HETATM':
        res = line[17:20]
        chain_id = line[21]
        res_num = line[22:26]
        res_num2 = line[22:27]
        atom_type = line[12:16]
        if chain_id in mod_res_dict:
            if res in mod_res_dict[chain_id] :
                protein_line_list += [line]
                continue
        if res not in ligand_line_dict:
            ligand_line_dict[res] = list()
        ligand_line_dict[res] += [line]
        
    if line[0:6] == 'CONECT':
        conect_list = []
        for i in range(0, 8):
            ini = i * 5 + 6
            fin = (i + 1) * 5 + 6
            atom_number = line[ini:fin].strip()
            if len(atom_number) > 0:
                conect_list += [int(atom_number)]
        conect_idx = conect_list[0]
        if conect_idx not in conect_dict:
            conect_dict[conect_idx] = conect_list[1:]
        else:
            conect_dict[conect_idx] = conect_dict[conect_idx] + conect_list[1:]

In [5]:
print(conect_dict)

{2356: [2357], 2357: [2356, 2358, 2362], 2358: [2357, 2359], 2359: [2358, 2360], 2360: [2359, 2361], 2361: [2360, 2362, 2363], 2362: [2357, 2361], 2363: [2361, 2364, 2378], 2364: [2363, 2365], 2365: [2364, 2372, 2379], 2366: [2367, 2371, 2379], 2367: [2366, 2368], 2368: [2367, 2369], 2369: [2368, 2370], 2370: [2369, 2371], 2371: [2366, 2370], 2372: [2365, 2373, 2377], 2373: [2372, 2374], 2374: [2373, 2375], 2375: [2374, 2376], 2376: [2375, 2377], 2377: [2372, 2376, 2378], 2378: [2363, 2377], 2379: [2365, 2366]}


In [18]:
#print(ligand_line_dict)

In [6]:

protein_file = 'pdb/3HMMA_receptor.pdb'
fp_p = open(protein_file, 'w')

molecule_atom_numbers = list()
for line in protein_line_list :
    fp_p.write(line)
    atom_number = int(line[6:11])
    molecule_atom_numbers += [atom_number]
line_out = 'TER\n'
fp_p.write(line_out)
for atom_number in molecule_atom_numbers:
    if atom_number not in conect_dict:
        continue
    ans = conect_dict[atom_number]
    ans2 = list()
    for an in ans:
        if an in molecule_atom_numbers:
            ans2 += [an]
    num_conect = len(ans2)
    line_out = ''
    for i_con in range(num_conect):
        if i_con % 4 == 0:
            line_out += 'CONECT%5d' % (atom_number)
        line_out += '%5d' % (ans2[i_con])
        if i_con % 4 == 3:
            line_out += '\n'
    if len(line_out.strip()) < 1:
        continue
    if line_out[-1] != '\n':
        line_out += '\n'
    fp_p.write(line_out)
fp_p.close()
    

In [7]:

ligand_name_list = ligand_line_dict.keys()
for ligand_name in ligand_name_list:
#    if ligand_name == 'HOH':
#        continue
    ligand_file = 'pdb/3HMMA_%s.pdb' %(ligand_name)
    fp_l = open(ligand_file, 'w')
    ligand_line_list = ligand_line_dict[ligand_name]
    molecule_atom_numbers = list()
    for line in ligand_line_list :
        fp_l.write(line)
        atom_number = int(line[6:11])
        molecule_atom_numbers += [atom_number]
        
    for atom_number in molecule_atom_numbers:
        if atom_number not in conect_dict:
            continue
        ans = conect_dict[atom_number]
        ans2 = list()
        for an in ans:
            if an in molecule_atom_numbers:
                ans2 += [an]
        num_conect = len(ans2)
        line_out = ''
        for i_con in range(num_conect):
            if i_con % 4 == 0:
                line_out += 'CONECT%5d' % (atom_number)
            line_out += '%5d' % (ans2[i_con])
            if i_con % 4 == 3:
                line_out += '\n'
        if len(line_out.strip()) < 1:
            continue
        if line_out[-1] != '\n':
            line_out += '\n'
        fp_l.write(line_out)
    fp_l.close()
